In [1]:
from officelib.xllib import *
from officelib import const

In [2]:
def addr(c): 
    return c.GetAddress(False, False)

def sum_pulses(top_cell):
    c = top_cell
    sum_cells = []
    while True:
        c = c.End(const.xlDown)
        if c.Offset(1, 0).Value is None:
            break  # Ran out of data 
        if c.Offset(2, 1).Value is None:
            continue  # skip stray noise
        start = c
        c = c.End(const.xlDown)
        if c.Offset(1,0).Value is None:
            break
        fcell = start.Offset(1, 2)
        form = "=sum(%s:%s)" % (addr(start), addr(c))
        fcell.Value = form
        sum_cells.append(fcell)
    return sum_cells

In [3]:
def make_pulse_sum_table(header_cell, sum_cells):
    c = header_cell
    c.Value = "Pulse Sums"
    c.EntireColumn.AutoFit()
    for s in sum_cells:
        c = c.Offset(2, 1)
        c.Value = "=" + addr(s)

In [4]:
def find_baseline(xl, cells):
    d_top = cells(1, 2)
    d_end = d_top.End(const.xlDown)
    d_rng = cells.Range(d_top, d_end)
    
    wsf = xl.WorksheetFunction
    mode = wsf.Mode(d_rng)
    min_ = wsf.Min(d_rng)
    
    if mode > 0.1:
        # maybe mode doesn't reflect value well?
        base = min_
    else:
        base = mode
    return base

def correct_baseline(xl, cells):
    base = find_baseline(xl, cells)
    d = cells(1, 2)
    c = d.Offset(1, 2)
    c.Value = "=%s-%f" % (addr(d), base)
    tgt = cells.Range(d.Offset(1, 2), d.End(const.xlDown).Offset(1, 2))
    c.AutoFill(tgt)
    

In [35]:
def find_pulses(ws, tl):
    
    rng = ws.Cells.Range(tl, tl.End(const.xlDown))
    data = rng.Value
    to_paste = []
    tl_col = tl.Column
    for row, (d,) in enumerate(data, 1):
        if d > 0:
            form = "=" + cellStr(row, tl_col)
        else:
            form = None
        to_paste.append((form,))
    tgt = ws.Cells.Range(tl.Offset(1, 2), tl.End(const.xlDown).Offset(1, 2))
    tgt.Value = to_paste
                
def autofill_range(c):
    return c.Worksheet.Cells.Range(c.Offset(1,2), 
                                   c.End(const.xlDown).Offset(1,2))
    

In [31]:
def shrink_markers(ob, sz=2):
    try:
        sc = ob.SeriesCollection()
    except AttributeError:
        ob.MarkerSize = sz
        return
    else:
        for s in sc:
            s.MarkerSize = sz

def make_chart(ws, x1, y1, title, x_ax, y_ax):
    chart = CreateChart(ws)
    cr = ws.Cells.Range
    x = cr(x1, x1.End(const.xlDown))
    y = cr(y1, y1.End(const.xlDown))
    CreateDataSeries(chart, x, y)
    FormatChart(chart, None, title, x_ax, y_ax, False, False)
    shrink_markers(chart)
    return chart

In [32]:
def analyze_sheet(xl, ws):
    cells = ws.Cells
    correct_baseline(xl, cells)
    pulse_cell = cells(1, 3)
    find_pulses(ws, pulse_cell)
    pulses = sum_pulses(pulse_cell.Offset(1, 2))
    make_pulse_sum_table(pulse_cell.Offset(1, 3), pulses)
    make_chart(ws, cells(1,1), cells(1,3), "CO2 Pulse Accuracy",
              "Time(ms)", "Flow Rate(L/min)")

In [36]:
xl = Excel()
def run():
    with HiddenXl(xl):
        analyze_sheet(xl, xl.ActiveSheet)
run()